# Preprocessing

In [1]:
# This Python file uses the following encoding: utf-8
import io
import os
import sys
import PyPDF2
from pathlib2 import Path
from shutil import copyfile
import textract
import lib.parallel_run
from lib.words import splitted_words, replace_digits

Sposta i files che soddisfano un dato criterio da un albero di sottocartelle ad una cartella

In [2]:
def move_flattened_files(src_dir, out_dir, filt):
    p = Path(src_dir)
    if p.is_dir():
        for f in p.iterdir():
            if f.is_file() and filt(f):
                copyfile(str(f.absolute()),str(out_dir + u'/' + f.name))
            else:
                move_flattened_files(str(f.absolute()), out_dir, filt)

def file_filter_cost(f):
    return f.name.endswith(u'001.pdf') and os.path.getsize(str(f.absolute()))<(1024**2)
    
def file_filter_non_cost(f):
    return os.path.getsize(str(f.absolute()))<(1024**2)

In [3]:
move_flattened_files('../raw_c','../atti_costitutivi', file_filter_cost)

In [5]:
move_flattened_files(u'../raw_a05',u'../verbali_a05/', file_filter_non_cost)

Toglie i files con un certo criterio (dimensione maggiore al MB)

In [3]:
def delete_files(dir_name, pred):
    p = Path(dir_name)
    for f in p.iterdir():
        if (not f.is_dir()) and pred(str(f.absolute())):
            print str(f.absolute())
            os.remove(str(f.absolute()))

In [ ]:
delete_files('../atti_costitutivi', lambda f: os.path.getsize(f)>(1024**2))

Utils

In [8]:
scans_filtered = [f for f in os.listdir("../ocr/scans_non_cost")]
#scans_filtered

Separa le scansioni di doc non costitutivi e le sposta in un'altra cartella

In [ ]:
for i in scans_filtered:
    os.rename('../ocr/scans/'+i+'.pdf', '../ocr/scans_non_cost/'+i+'.pdf')
    print i + '.pdf'

Converte i pdf in file di testo. sposta i fallimenti in altre cartelle.

In [7]:
def extract_text(f,out_dir, scans_dir, unhandled_dir, minlen):
    file = str(f.absolute())
    #print f.name
    try:
        text = textract.process(file).strip()
        if len(splitted_words(text)) <= minlen:
            print f.name, "is empty"
            copyfile(file, str(scans_dir + '/' + f.name))
        else:
            with open(out_dir + "/" + f.name[:-3] + 'txt', 'w') as outf:
                outf.write(text)
    except:
        print f.name
        copyfile(file, str(unhandled_dir + '/' + f.name))

def read_pdf_content(src_dir, out_dir, scans_dir, unhandled_dir, minlen):
    p = Path(src_dir)
    lib.parallel_run.parallel_run_args(extract_text,((f, out_dir, scans_dir, unhandled_dir, minlen) for f in p.iterdir()))
    
    
def read_pdf_content_old(src_dir, out_dir, scans_dir, unhandled_dir, minlen):
    p = Path(src_dir)
    for f in p.iterdir():
        extract_text(f,out_dir, scans_dir, unhandled_dir, minlen)

In [9]:
read_pdf_content("../atti_costitutivi", "../qq", "../ocr/scans_cost/", "../ocr/unhandled_files", 200)

4935800480001.pdf is empty
4970121470001.pdf is empty
4948718160001.pdf
5052664760001.pdf is empty
5063014890001.pdf is empty
5005268220001.pdf
4908476390001.pdf is empty
5032797020001.pdf is empty
5031576180001.pdf is empty
5024059580001.pdf
5098102360001.pdf is empty
5035902370001.pdf is empty
4976122080001.pdf is empty
5026105640001.pdf is empty
4971402220001.pdf is empty
5101055230001.pdf is empty
4953368670001.pdf is empty
5058500750001.pdf is empty
5122092080001.pdf is empty
5066201520001.pdf is empty
5084617720001.pdf is empty
5093915080001.pdf
4956431810001.pdf is empty
4997045230001.pdf is empty
4931826640001.pdf is empty
4980613210001.pdf is empty
5082985060001.pdf is empty
5042230110001.pdf
4959437960001.pdf
5072496540001.pdf is empty
4951231570001.pdf is empty
4966154440001.pdf is empty
4910714930001.pdf is empty
4932578270001.pdf is empty
5081809570001.pdf is empty
5022150630001.pdf
4962756680001.pdf is empty
4992573520001.pdf is empty
5064812960001.pdf is empty
5003665660

In [10]:
read_pdf_content('../atti_non_costitutivi','../atti_non_costitutivi_txt','../ocr/scans_non_cost','../ocr/unhandled_files', 100)

4120010350004.pdf is empty
3529457650003.pdf is empty
3769293200002.pdf is empty
4299820700004.pdf is empty
4505787470001.pdf is empty
4518841450002.pdf is empty
3917207730001.pdf is empty
4941837870004.pdf is empty
4652280370009.pdf is empty
3703508720002.pdf is empty
4104374780003.pdf is empty
4714406240001.pdf is empty
4860752530001.pdf is empty
3667791020001.pdf is empty
4997803750001.pdf is empty
4683987650001.pdf is empty
4932792600003.pdf is empty
4107251120002.pdf is empty
4109400810008.pdf is empty
4202672400004.pdf is empty
3335133670002.pdf is empty
3507044040003.pdf is empty
4560302190001.pdf is empty
4421722810001.pdf is empty
3883639860005.pdf is empty
4288659810003.pdf is empty
3808342640003.pdf is empty
4103724410003.pdf is empty
4255716820005.pdf is empty
3667352930001.pdf is empty
3833332710001.pdf is empty
4612855460002.pdf is empty
4689247370002.pdf is empty
4837623220002.pdf is empty
3468589000003.pdf is empty
3564752420003.pdf is empty
5078271310002.pdf is empty
4

In [8]:
read_pdf_content('../verbali_a05/','../verbali_a05_txt/','../ocr/scans_non_cost','../ocr/unhandled_files', 100)

5162881130008.pdf is empty
5162600560002.pdf is empty
5166106220006.pdf is empty
5169861960002.pdf is empty
5167582400001.pdf is empty
5173669880002.pdf is empty
5180767010001.pdf is empty
5184378270001.pdf is empty
5183713540002.pdf is empty
5167734950003.pdf is empty
5177240440001.pdf is empty
5176266390006.pdf is empty
5182281390004.pdf is empty
5183092070002.pdf is empty
5173441580002.pdf is empty
5171756340002.pdf is empty
5172995190002.pdf is empty
5165490860002.pdf is empty
5177815010004.pdf is empty
5169840280003.pdf is empty
5166681440004.pdf is empty
5168635120003.pdf is empty
5167827320001.pdf is empty
5168911610001.pdf is empty
5169600160003.pdf is empty
5167182630001.pdf
5179420560002.pdf is empty
5166817090001.pdf is empty
5163027230003.pdf is empty
5170614520001.pdf is empty
5176259580001.pdf is empty
5174444900002.pdf is empty
5182425390001.pdf
5167734820003.pdf is empty
5172922700002.pdf is empty
5166723350002.pdf is empty
5163117100002.pdf is empty
5176199210003.pdf i

# GOOGLE VISION API 

Extract jpg's from pdf's. Quick and dirty.

In [19]:
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

import codecs

### New Version

In [58]:
import lib.text_extraction as te

import subprocess
from lib.utils import rm_dir

def ocr_and_text(f, png_dir, pages, min_pages):
    rm_dir(png_dir)
    os.mkdir(png_dir)
    # out-10.png
    res = subprocess.check_output(("convert -verbose -density 300 %s -quality 100 %s" 
          % (os.path.join("../ocr/scans_cost", f), os.path.join(png_dir,os.path.basename(f)[:-3]+'png'))).split(' '))
    images = os.listdir(png_dir)
    
    if len(images) > min_pages:
        images = sorted(images, key=lambda item: (int(item.partition('-')[2].partition('.')[0])))
        res = ""
        for i,el in enumerate(images):
            if i < pages:
                txt = te.convert_to_txt(os.path.join(png_dir,el))
                res += txt
            else:
                break
        rm_dir(png_dir)        
        if res != "":
            return res
        else:
            print f, "is empty"
            return ''
    else:
        return ''

In [ ]:
for f in os.listdir("../ocr/scans_cost/"):
    txt = ocr_and_text(f, '../ocr/tmp', 6, 2)
    if txt != "":
        print f
        with codecs.open("../ocr/scanned_costitutivi/" + f[:-4] + ".txt", "wb", 'utf-8') as outf:        
            outf.write(u'' + txt)

5052664760001.pdf
5063014890001.pdf
4908476390001.pdf
5035902370001.pdf
5026105640001.pdf
4971402220001.pdf
5122092080001.pdf
5066201520001.pdf
4956431810001.pdf
4980613210001.pdf
5082985060001.pdf
4910714930001.pdf
5081809570001.pdf
4962756680001.pdf
4992573520001.pdf
5003665660001.pdf
5005263490001.pdf
5120587790001.pdf
5063020700001.pdf
5027317680001.pdf
4992675340001.pdf
4983494400001.pdf
4978390410001.pdf
4922874250001.pdf
5096330640001.pdf
5051903310001.pdf
4966881100001.pdf
5006551660001.pdf
4955799250001.pdf
4980607140001.pdf
4952787610001.pdf
5095081520001.pdf
5068374490001.pdf
5091109730001.pdf
5014839980001.pdf
5090950190001.pdf
5031766730001.pdf
4976484670001.pdf
5109549230001.pdf
5050977560001.pdf
5034662830001.pdf
4934877950001.pdf
4925471890001.pdf
5029566790001.pdf
4998884750001.pdf
4979145690001.pdf
5029030220001.pdf
5119293140001.pdf
4937749680001.pdf
4982235420001.pdf
4965090420001.pdf
4971318170001.pdf
5015181940001.pdf
4946463520001.pdf
4932081580001.pdf
5011524910

In [ ]:
i = 0
for f in os.listdir("../ocr/scans_non_cost")):
    if i >= len(os.listdir("../ocr/scanned_costitutivi")):
        break         
    txt = ocr_and_text(f, '../ocr/tmp', 5, 0)
    if txt != "":
        print f
        i += 1
        with codecs.open("../ocr/scanned_non_costitutivi/" + f[:-4] + ".txt", "wb", 'utf-8') as outf:        
            outf.write(u'' + txt)

In [26]:
call(("convert -verbose -density 300 %s -quality 100 %s" 
          % ("../ocr/scans_cost/4935800480001.pdf", os.path.join('../ocr/tmp',"4935800480001.pdf"[:-3]+'png'))).split(' '))

0

### Old Version

In [14]:
def convert_to_pdf(f, res):
    pdf = file("../ocr/scans_non_cost2/" + f, "rb").read()

    startmark = "\xff\xd8"
    startfix = 0
    endmark = "\xff\xd9"
    endfix = 2
    i = 0

    njpg = 0
    while True:
        istream = pdf.find("stream", i)
        if istream < 0:
            break
        istart = pdf.find(startmark, istream, istream+20)
        if istart < 0:
            i = istream+20
            continue
        iend = pdf.find("endstream", istart)
        if iend < 0:
            return
            raise Exception("Didn't find end of stream!")
        iend = pdf.find(endmark, iend-20)
        if iend < 0:
            return
            raise Exception("Didn't find end of JPG!")

        istart += startfix
        iend += endfix
        #print "JPG %d from %d to %d" % (njpg, istart, iend)
        res.append(pdf[istart:iend])
        #jpgfile = file("ocr/tempjpg/" + f[:-4] + "_%d.jpg" % njpg, "wb")
        #jpgfile.write(jpg)
        #jpgfile.close()
        #print ("ocr/tempjpg/" + f[:-4] + "_%d.jpg" % njpg)

        #njpg += 1
        i = iend

In [8]:
# Instantiates a client
client = vision.ImageAnnotatorClient()

def convert_to_txt(img):
    image = types.Image(content=img)
    # Performs label detection on the image file
    response = client.text_detection(image=image)
    labels = response.text_annotations
    
    try:
        return (labels[0].description)
    except:
        return ""   

In [9]:
for i,f in enumerate(os.listdir("../ocr/scans_non_cost2")):
    print i
    images = []
    convert_to_pdf(f, images)
    if len(images) > 0:
        res = ""
        for el in images:
            txt = convert_to_txt(el)
            res += txt
        if res != "":        
            with codecs.open("../ocr/scanned_non_costitutivi/" + f[:-4] + ".txt", "wb", 'utf-8') as outf:        
                outf.write(u'' + res)
        else:
            print f + " is empty"

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
3632943460001.pdf is empty
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
3626146800001.pdf is empty
121
122
4060862800001.pdf is empty
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
4113279830001.pdf is empty
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
3483842640001.pdf is empty
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243

## Test area

In [18]:
def convert_pdf():
    pdf = file("../files_to_predict/4908312250001.pdf", "rb").read()

    startmark = "\xff\xd8"
    startfix = 0
    endmark = "\xff\xd9"
    endfix = 2
    i = 0

    njpg = 0
    while njpg < 5:
        istream = pdf.find("stream", i)
        if istream < 0:
            print "istream < 0"
            break
        istart = pdf.find(startmark, istream, istream+20)
        if istart < 0:
            i = istream+20
            continue
        iend = pdf.find("endstream", istart)
        if iend < 0:
            print "iend < 0"
            raise Exception("Didn't find end of stream!")
        iend = pdf.find(endmark, iend-20)
        if iend < 0:
            print "iend < 0"
            raise Exception("Didn't find end of JPG!")

        istart += startfix
        iend += endfix
        print "JPG %d from %d to %d" % (njpg, istart, iend)
        jpg = pdf[istart:iend]
        jpgfile = file("../files_to_predict/jpg%d.jpg" % njpg, "wb")
        jpgfile.write(jpg)
        jpgfile.close()

        njpg += 1
        i = iend

# Instantiates a client
client = vision.ImageAnnotatorClient()

def convert_txt(image):
    # Loads the image into memory
    with io.open(image, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    # Performs label detection on the image file
    response = client.text_detection(image=image)
    labels = response.text_annotations

    print (labels[0].description)
        

In [20]:
convert_pdf()

istream < 0


In [16]:
convert_txt("../files_to_predict/to-0.png")

(riuseppe Muria Miceli
N. 365 Repertorido
N. 179 Raccolta
ATTO COSTITUTIVO DI SOCIETA
A RESPONSABILITA' LIMITATA SEMPLIFICATA
REPUBBLICA ITALIANA
Il giorno sette febbraio duemiladiciassette
07.02,2017
In Sesto San Giovanni, nel mio studio, alla via Felice Cavallotti n
Davanti a me Dott. Avv. Giuseppe Maria Miceli, Notaio iscritto
presso il Collegio Notarile dei Distretti Notarili Riuniti di Milano,
Busto Arsizio, Lodi, Monza e Varese, con sede in Sesto San Gio-Registrato u Milano 6
vanni
OFFICIO DELLE FNTRATE
SI SONO COSTITUITI
Puglisi Salvatore Antonio, nato a Milano il 28 settembre 1969, IL08/02/2017
residente Milano, via Edolo 5,codice fiscale:, N.4408
PGLSVT69P28F205B, che dichiara di essere coniugato in regime
di separazione dei beni;
Puglisi Nicolò, nato a Genova il 03 febbraio 1999, residente al versaTI C 200,00
Cogorno (GE), via A Castello 41, codice fiscale:
PGLNCL99B03D969A, che dichiara di essere di stato civile libero;
Sanfilippo Luca, nato a Roma il 24 maggio 1970, residen